### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.6/759.6 KB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.

### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Using custom data configuration SetFit--qqp-e0faace7f63208a7
Found cached dataset json (C:/Users/pavel.sokolov/.cache/huggingface/datasets/SetFit___json/SetFit--qqp-e0faace7f63208a7/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 3/3 [00:00<00:00, 99.15it/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\pavel.sokolov\.cache\huggingface\datasets\SetFit___json\SetFit--qqp-e0faace7f63208a7\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-d604221af1b8f370.arrow
100%|██████████| 391/391 [00:47<00:00,  8.30ba/s]
Loading cached processed dataset at C:\Users\pavel.sokolov\.cache\huggingface\datasets\SetFit___json\SetFit--qqp-e0faace7f63208a7\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-a06b1485b2ae01f6.arrow


In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [ ]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
import evaluate


accuracy_metric = evaluate.load("accuracy")
accuracy_metric.compute(references=[0, 1], predictions=[0, 1])


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = model.to(device)

BATCH = 256
WORKERS = 2

val_set = qqp_preprocessed['validation']

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=BATCH,
    shuffle=False,
    collate_fn=transformers.default_data_collator,
    num_workers=WORKERS,
)
len(val_set)

cpu


40430

In [ ]:
%%time
all_predictions = torch.tensor([]).to(device)
all_labels = torch.tensor([]).to(device)

for i, batch in enumerate(val_loader):
    # batch = batch.to(device)
    # batc
    with torch.no_grad():
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )
        predictions = torch.argmax(predicted.logits, dim=1).to(device)
        all_predictions = torch.cat([all_predictions, predictions]).to(device)
        all_labels = torch.cat([all_labels, batch['labels'].to(device)]).to(device)
    if i % 10 == 0 and i != 0:
        print(f'{i} iter')

results = accuracy_metric.compute(references=all_labels, predictions=all_predictions)
print(results)

10 iter
20 iter
30 iter
40 iter
50 iter
60 iter
70 iter
80 iter
90 iter
100 iter
110 iter
120 iter
130 iter
140 iter
150 iter
{'accuracy': 0.9083848627256987}
CPU times: total: 5h 34min 28s
Wall time: 1h 1min 56s


In [ ]:
accuracy = results['accuracy']
print('acc =', accuracy)
assert 0.9 < accuracy < 0.91

acc = 0.9083848627256987


### Task 2: train the model (5 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

qqp = datasets.load_dataset('SetFit/qqp')

cuda:0


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/SetFit___json/SetFit--qqp-e0faace7f63208a7/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import Trainer, TrainingArguments


model_name = "funnel-transformer/small"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of the model checkpoint at funnel-transformer/small were not used when initializing FunnelForSequenceClassification: ['decoder.layers.1.attention.layer_norm.bias', 'decoder.layers.1.attention.v_head.weight', 'decoder.layers.1.attention.post_proj.bias', 'decoder.layers.1.ffn.linear_2.bias', 'decoder.layers.1.attention.r_kernel', 'decoder.layers.1.attention.r_s_bias', 'decoder.layers.0.attention.post_proj.weight', 'decoder.layers.0.attention.v_head.bias', 'decoder.layers.0.attention.v_head.weight', 'decoder.layers.0.ffn.linear_2.weight', 'decoder.layers.0.ffn.linear_2.bias', 'decoder.layers.0.attention.layer_norm.weight', 'decoder.layers.0.attention.r_kernel', 'decoder.layers.0.ffn.layer_norm.bias', 'decoder.layers.0.attention.q_head.weight', 'decoder.layers.1.attention.v_head.bias', 'decoder.layers.0.ffn.linear_1.weight', 'decoder.layers.0.attention.k_head.bias', 'decoder.layers.1.attention.k_head.weight', 'decoder.layers.1.attention.post_proj.weight', 'decoder.layers.1.att

In [ ]:
MAX_LENGTH = 128

def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

  0%|          | 0/364 [00:00<?, ?ba/s]

  0%|          | 0/391 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

In [ ]:
BATCH = 64
WORKERS = 1
EPOCHS = 1

training_args = TrainingArguments(output_dir='sample_data',
                                  dataloader_num_workers=WORKERS,
                                  per_device_train_batch_size=BATCH,
                                  num_train_epochs=EPOCHS,
                                  )

In [ ]:
torch.cuda.empty_cache()
import gc
# del variables
gc.collect()

53

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 11.8 MB/s eta 0:00:00


In [ ]:
import evaluate
import numpy as np


accuracy_metric = evaluate.load("accuracy")


def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

torch.cuda.empty_cache()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=qqp_preprocessed['train'],
    eval_dataset=qqp_preprocessed['validation'],
    compute_metrics=compute_accuracy,
    tokenizer=tokenizer,
    data_collator=transformers.default_data_collator
)
# Training
train_result = trainer.train()

The following columns in the training set don't have a corresponding argument in `FunnelForSequenceClassification.forward` and have been ignored: text1, label_text, idx, text2. If text1, label_text, idx, text2 are not expected by `FunnelForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 363846
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 5686
  Number of trainable parameters = 116203778


Step,Training Loss
500,0.393400
1000,0.340200
1500,0.318900
2000,0.299700
2500,0.292800
3000,0.277500
3500,0.262900
4000,0.266400
4500,0.247700
5000,0.242400


Saving model checkpoint to sample_data/checkpoint-500
Configuration saved in sample_data/checkpoint-500/config.json
Model weights saved in sample_data/checkpoint-500/pytorch_model.bin
tokenizer config file saved in sample_data/checkpoint-500/tokenizer_config.json
Special tokens file saved in sample_data/checkpoint-500/special_tokens_map.json
Saving model checkpoint to sample_data/checkpoint-1000
Configuration saved in sample_data/checkpoint-1000/config.json
Model weights saved in sample_data/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in sample_data/checkpoint-1000/tokenizer_config.json
Special tokens file saved in sample_data/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to sample_data/checkpoint-1500
Configuration saved in sample_data/checkpoint-1500/config.json
Model weights saved in sample_data/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in sample_data/checkpoint-1500/tokenizer_config.json
Special tokens file saved in sample_dat

In [ ]:
# save model
trainer.save_model(f"trained_{model_name.replace('/', '_')}")

Saving model checkpoint to trained_funnel-transformer_small
Configuration saved in trained_funnel-transformer_small/config.json
Model weights saved in trained_funnel-transformer_small/pytorch_model.bin
tokenizer config file saved in trained_funnel-transformer_small/tokenizer_config.json
Special tokens file saved in trained_funnel-transformer_small/special_tokens_map.json


### Task 3: try the full pipeline (2 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [ ]:
import torch
import transformers
import datasets
import numpy as np

C:\Users\pavel.sokolov\Miniconda3\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load model and dataset preprocessing
model_name = "funnel-transformer/small"
model_path = f"trained_funnel-transformer_small/"
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_path, )
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
qqp = datasets.load_dataset('SetFit/qqp')

MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Using custom data configuration SetFit--qqp-e0faace7f63208a7
Found cached dataset json (C:/Users/pavel.sokolov/.cache/huggingface/datasets/SetFit___json/SetFit--qqp-e0faace7f63208a7/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 3/3 [00:00<00:00,  7.39it/s]
Loading cached processed dataset at C:\Users\pavel.sokolov\.cache\huggingface\datasets\SetFit___json\SetFit--qqp-e0faace7f63208a7\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-d2e7e0b3412ecad0.arrow
Loading cached processed dataset at C:\Users\pavel.sokolov\.cache\huggingface\datasets\SetFit___json\SetFit--qqp-e0faace7f63208a7\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-83292120184ad590.arrow
Loading cached processed dataset at C:\Users\pavel.sokolov\.cache\huggingface\datasets\SetFit___json\SetFit--qqp-e0faace7f63208a7\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-092831b04b444ac0.arrow


In [ ]:
# validation parameters

BATCH = 1024
WORKERS = 2
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=BATCH,
    shuffle=False,
    collate_fn=transformers.default_data_collator,
    num_workers=WORKERS,
)#%% md
### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

### Task 1: evaluation (1 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [ ]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
import evaluate


accuracy_metric = evaluate.load("accuracy")
accuracy_metric.compute(references=[0, 1], predictions=[0, 1])


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = model.to(device)

BATCH = 256
WORKERS = 2

val_set = qqp_preprocessed['validation']

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=BATCH,
    shuffle=False,
    collate_fn=transformers.default_data_collator,
    num_workers=WORKERS,
)
len(val_set)

In [ ]:
%%time
all_predictions = torch.tensor([]).to(device)
all_labels = torch.tensor([]).to(device)

for i, batch in enumerate(val_loader):
    # batch = batch.to(device)
    # batc
    with torch.no_grad():
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )
        predictions = torch.argmax(predicted.logits, dim=1).to(device)
        all_predictions = torch.cat([all_predictions, predictions]).to(device)
        all_labels = torch.cat([all_labels, batch['labels'].to(device)]).to(device)
    if i % 10 == 0 and i != 0:
        print(f'{i} iter')

results = accuracy_metric.compute(references=all_labels, predictions=all_predictions)
print(results)

In [ ]:
accuracy = results['accuracy']
print('acc =', accuracy)
assert 0.9 < accuracy < 0.91

### Task 2: train the model (5 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

qqp = datasets.load_dataset('SetFit/qqp')

In [ ]:
from transformers import Trainer, TrainingArguments


model_name = "funnel-transformer/small"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
MAX_LENGTH = 128

def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [ ]:
BATCH = 64
WORKERS = 1
EPOCHS = 1

training_args = TrainingArguments(output_dir='sample_data',
                                  dataloader_num_workers=WORKERS,
                                  per_device_train_batch_size=BATCH,
                                  num_train_epochs=EPOCHS,
                                  )

In [ ]:
torch.cuda.empty_cache()
import gc
# del variables
gc.collect()

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
import numpy as np


accuracy_metric = evaluate.load("accuracy")


def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

torch.cuda.empty_cache()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=qqp_preprocessed['train'],
    eval_dataset=qqp_preprocessed['validation'],
    compute_metrics=compute_accuracy,
    tokenizer=tokenizer,
    data_collator=transformers.default_data_collator
)
# Training
train_result = trainer.train()

In [ ]:
# save model
trainer.save_model(f"trained_{model_name.replace('/', '_')}")

### Task 3: try the full pipeline (2 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [21]:
import torch
import transformers
import datasets
import numpy as np
import evaluate

In [16]:
# load model and dataset preprocessing
model_name = "funnel-transformer/small"
model_path = f"trained_funnel-transformer_small/"
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_path, )
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
qqp = datasets.load_dataset('SetFit/qqp')

MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/SetFit___json/SetFit--qqp-e0faace7f63208a7/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

In [17]:
# validation parameters
BATCH = 1024
WORKERS = 2
val_set = qqp_preprocessed['validation']
# load the data
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=BATCH,
    shuffle=False,
    collate_fn=transformers.default_data_collator,
    num_workers=WORKERS,
)

In [22]:
# check accuracy
accuracy_metric = evaluate.load("accuracy")
all_predictions = torch.tensor([]).to(device)
all_labels = torch.tensor([]).to(device)

for i, batch in enumerate(val_loader):
    with torch.no_grad():
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )
        predictions = torch.argmax(predicted.logits, dim=1).to(device)
        all_predictions = torch.cat([all_predictions, predictions]).to(device)
        all_labels = torch.cat([all_labels, batch['labels'].to(device)]).to(device)

results = accuracy_metric.compute(references=all_labels, predictions=all_predictions)
print(results)

{'accuracy': 0.9005441503833786}


__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.